## LabJack Example\
Demonstrates reading 2 analog inputs (AINs) in a loop from a LabJack.

Relevant Documentation:
 
LJM Library:
    LJM Library Installer:
        https://labjack.com/support/software/installers/ljm
    LJM Users Guide:
        https://labjack.com/support/software/api/ljm
    Opening and Closing:
        https://labjack.com/support/software/api/ljm/function-reference/opening-and-closing
    Multiple Value Functions(such as eWriteNames):
        https://labjack.com/support/software/api/ljm/function-reference/multiple-value-functions
    Timing Functions(such as StartInterval):
        https://labjack.com/support/software/api/ljm/function-reference/timing-functions
 
T-Series and I/O:
    Modbus Map:
        https://labjack.com/support/software/api/modbus/modbus-map
    Analog Inputs:
        https://labjack.com/support/datasheets/t-series/ain

In [ ]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

from labjack import ljm

In [ ]:
# Open first found LabJack
handle = ljm.openS("T7", "ANY", "ANY")  # T7 device, Any connection, Any identifier

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Open first found LabJack
handle = ljm.openS("T7", "ethernet", "139.229.191.31")  # T7 device, Any connection, Any identifier

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Setup and call eWriteNames to configure AIN0 and AIN1 on the LabJack.
# LabJack T7 and other devices configuration

# AIN0 and AIN1:
#   Negative channel = single ended (199)
#   Range: +/-10.0 V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0)
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US"]
aValues = [199, 10.0, 0, 0,
           199, 10.0, 0, 0]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

In [ ]:
print("\nSet configuration:")
for i in range(numFrames):
    print("    %s : %f" % (names[i], aValues[i]))


In [ ]:
loopAmount = 10
loopMessage = "READ"
# Read AIN0 and AIN1 from the LabJack with eReadNames in a loop.
numFrames = 2
names = ["AIN0", "AIN1"]

print("\nStarting %s read loops.%s\n" % (str(loopAmount), loopMessage))
intervalHandle = 1
ljm.startInterval(intervalHandle, 10000)  # Delay between readings (in microseconds)
i = 0
while True:
    try:
        results = ljm.eReadNames(handle, numFrames, names)
        print("AIN0 : %f V, AIN1 : %f V" % (results[0], results[1]))
        ljm.waitForNextInterval(intervalHandle)
        if loopAmount != "infinite":
            i = i + 1
            if i >= loopAmount:
                break
    except KeyboardInterrupt:
        break
    except Exception:
        print(sys.exc_info()[1])
        break


In [ ]:
# Setup and call eWriteName to write a value to the LabJack.
name = "DAC0"
value = 1.73  # 2.5 V
ljm.eWriteName(handle, name, value)

print("\neWriteName: ")
print("    Name - %s, value : %f" % (name, value))

In [ ]:
loopAmount = 1000
loopMessage = "READ"
# Read AIN0 and AIN1 from the LabJack with eReadNames in a loop.
numFrames = 2
names = ["AIN0", "AIN1"]

print("\nStarting %s read loops.%s\n" % (str(loopAmount), loopMessage))
intervalHandle = 1
ljm.startInterval(intervalHandle, 10000)  # Delay between readings (in microseconds)
i = 0

plotResults = []

while True:
    try:
        results = ljm.eReadNames(handle, numFrames, names)
        plotResults.append(results)
        #print("AIN0 : %f V, AIN1 : %f V" % (results[0], results[1]))
        ljm.waitForNextInterval(intervalHandle)
        if loopAmount != "infinite":
            i = i + 1
            if i >= loopAmount:
                break
    except KeyboardInterrupt:
        break
    except Exception:
        print(sys.exc_info()[1])
        break


In [ ]:
plt.plot(np.array(plotResults)[10:-1,0], color='red')
plt.plot(np.array(plotResults)[10:-1,1], color='blue')

In [ ]:
# Play with ramping up DAC0 and reading value with AIN0.
name = "DAC0"
loopAmount = 1100
# Read AIN0 from the LabJack with eReadNames in a loop.
names = ["AIN0"]
numFrames = 1
intervalHandle = 1
ljm.startInterval(intervalHandle, 1000)  # Delay between readings (in microseconds)
plotResults = []
value = 0.0
ljm.eWriteName(handle, name, value)
for n in range(loopAmount):
    if n > 0 and n % 100 == 0:
        value += 0.5
        ljm.eWriteName(handle, name, value)
    results = ljm.eReadNames(handle, numFrames, names)
    plotResults.append(results)

plt.plot(np.array(plotResults)[:,0], color='red')

In [ ]:
# Close handles
ljm.close(handle)